In [164]:
import numpy as np
from scipy import interpolate
from tqdm import tqdm, trange
import seaborn as sns
from matplotlib import pyplot as plt
from torchvision.transforms import Resize
from sklearn.metrics import pairwise_distances
import umap
import torch
import os
import json
from barcodes import rtd_from_ats
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, matthews_corrcoef, accuracy_score
from sklearn.model_selection import cross_val_score, KFold
from scipy.stats import wasserstein_distance_nd
from catboost import CatBoostClassifier

In [165]:
heads = [
         16,   45,   56,   90,   93,  101,  117,  127,  136,  164,  165,
        205,  214,  223,  227,  231,  244,  250,  251,  260,  262,  267,
        274,  282,  283,  310,  336,  339,  351,  371,  373,  376,  379,
        380,  381,  401,  415,  416,  432,  450,  457,  461,  468,  469,
        477,  495,  501,  506,  511,  514,  549,  589,  607,  622,  626,
        629,  631,  632,  639,  648,  655,  657,  658,  664,  667,  668,
        675,  689,  690,  691,  703,  716,  740,  754,  785,  788,  805,
        818,  833,  837,  854,  862,  863,  865,  895,  898,  901,  918,
        927,  938,  956,  975,  976,  990,  991, 1003, 1009, 1011, 1015
    ]

def get_data(dirpath, TS=False):
    X, y = [], []
    for f in os.listdir(dirpath):
        if f[:2]!='ex' or len(f.split('.'))!=3 or f[-5]=='q': continue
        label = f.split('_')[1][0]
        crat = np.load(dirpath+f)
        if TS:
            crat = crat.reshape(32*32,15,2)[heads,:]#[:,:,[2,3]]#[:,:,[1,2,3]]
        else:
            crat = crat.reshape(99,15,2)#[:,[2,3]]
        X.append(crat.ravel())
        y.append(int(label))
    return np.array(X), np.array(y)

Xtr, ytr = get_data('auto-labeled/output/llamabase7b/train_ff/')
Xvl, yvl = get_data('auto-labeled/output/llamabase7b/valid_ff/')
Xts, yts = get_data('auto-labeled/output/llamabase7b/test_ff/')
XTS, yTS = get_data('helm/filtration_features/llamabase7b/', True)

mask = np.arange(ytr.shape[0])
mask = sorted(list( mask[ytr.astype(bool)][:(1-ytr).sum()] )+list(mask[~ytr.astype(bool)][:ytr.sum()]))
Xtr, ytr = Xtr[mask], ytr[mask]

In [169]:
# m = Pipeline([('ss',StandardScaler()),('est',LogisticRegression(max_iter=5000))])
m = Pipeline([('ss',StandardScaler()),('est',MLPClassifier((256,128,64),
        max_iter=100, batch_size=32, learning_rate_init=5e-4))])
# m = CatBoostClassifier(verbose=False)
m = m.fit(Xtr, ytr)
ypr = m.predict(Xtr)
print(roc_auc_score(ytr,ypr))
ypr = m.predict(Xvl)
print(roc_auc_score(yvl,ypr))
ypr = m.predict(Xts)
print(roc_auc_score(yts,ypr))
ypr = m.predict(XTS)
print(roc_auc_score(yTS,ypr))

0.9586648983200707
0.6659343434343434
0.6519575395544075
0.581358791503719


In [159]:
def get_data_auto(type_="train", root_path='auto-labeled/output'):
    hd = json.load(open(os.path.join(root_path, 'llamabase7b', f"last_token_mean_{type_}.json"), encoding='utf-8'))
    hd3 = json.load(open(os.path.join(root_path, 'llamabase7b', f"last_mean_{type_}.json"), encoding='utf-8'))
    halu_hd, right_hd, halu_hd3, right_hd3 = [],[],[],[]
    for hs, hs3 in zip(hd, hd3):
        halu_hd += hs["hallu"]
        halu_hd3 += hs3["hallu"]
        right_hd.append(hs["right"])
        right_hd3.append(hs3["right"])
    right_hd = right_hd[:len(halu_hd)]
    right_hd3 = right_hd3[:len(halu_hd)]
    X, y = [], []
    for i in range(len(halu_hd)):
        X.append(np.array(right_hd[i])+np.array(right_hd3[i]))
        X.append(np.array(halu_hd[i])+np.array(halu_hd3[i]))
        y.append(0)
        y.append(1)
    return np.array(X), np.array(y)

def get_data(dirpath, TS=False):
    X, y = [], []
    for f in os.listdir(dirpath):
        if f[:2]!='hd' or f[-3:]!='npy' or f[-5]=='q': continue
        label = f.split('_')[1][0]
        crat = np.load(dirpath+f)
        X.append(crat.ravel())
        y.append(int(label))
    return np.array(X), np.array(y)

Xtr, ytr = get_data_auto('train')
Xvl, yvl = get_data_auto('valid')
Xts, yts = get_data_auto('test')
XTS, yTS = get_data('helm/hd/llamabase7b/', True)

In [160]:
# m = Pipeline([('ss',StandardScaler()),('est',LogisticRegression(max_iter=5000))])
# m = Pipeline([('ss',StandardScaler()),('est',MLPClassifier((256,128,64),
#         max_iter=100, batch_size=32, learning_rate_init=5e-4))])
m = CatBoostClassifier(verbose=False, )
m = m.fit(Xtr, ytr)
ypr = m.predict(Xtr)
print(accuracy_score(ytr,ypr))
ypr = m.predict(Xvl)
print(accuracy_score(yvl,ypr))
ypr = m.predict(Xts)
print(accuracy_score(yts,ypr))
ypr = m.predict(XTS)
print(roc_auc_score(yTS,ypr))

0.9531388152077807
0.6944444444444444
0.7157894736842106
0.6572157043171537
